<a href="https://colab.research.google.com/github/tasnimislamraisa/Python_Learning/blob/deep-Learning/LipReading_Optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎥 Lip Reading with TensorFlow (Optimized with .npz Preprocessing)

In [1]:
!pip install gdown
!apt install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
import os
import cv2
import gdown
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from matplotlib import pyplot as plt

In [3]:
url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
gdown.download(url, 'data.zip', quiet=False)
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

Downloading...
From (original): https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL
From (redirected): https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL&confirm=t&uuid=87384ddf-e171-4e39-b7f9-7ba7dd0a935b
To: /content/data.zip
100%|██████████| 423M/423M [00:07<00:00, 53.6MB/s]


In [4]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True)

def preprocess_and_save(video_dir='./data/s1', align_dir='./data/alignments/s1', out_dir='./npz_data'):
    os.makedirs(out_dir, exist_ok=True)
    for file in os.listdir(video_dir):
        if not file.endswith('.mpg'):
            continue
        file_name = os.path.splitext(file)[0]
        video_path = os.path.join(video_dir, file)
        align_path = os.path.join(align_dir, f"{file_name}.align")

        cap = cv2.VideoCapture(video_path)
        frames = []
        for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            ret, frame = cap.read()
            frame = tf.image.rgb_to_grayscale(frame)
            frames.append(frame[190:236, 80:220, :])
        cap.release()
        frames = tf.cast(frames, tf.float32)
        frames = (frames - tf.math.reduce_mean(frames)) / tf.math.reduce_std(frames)

        with open(align_path, 'r') as f:
            lines = f.readlines()
        tokens = []
        for line in lines:
            line = line.split()
            if line[2] != 'sil':
                tokens.extend([' ', line[2]])
        alignments = char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

        np.savez_compressed(os.path.join(out_dir, f"{file_name}.npz"),
                            frames=frames.numpy(), alignments=alignments.numpy())




In [5]:
import os

print("Current working directory:", os.getcwd())
print("Contents:", os.listdir())


Current working directory: /content
Contents: ['.config', 'data.zip', 'data', 'sample_data']


In [6]:
import os

for root, dirs, files in os.walk("data"):
    print("📁", root)
    for d in dirs:
        print("   └──", d)


📁 data
   └── data
📁 data/data
   └── s1
   └── alignments
📁 data/data/s1
📁 data/data/alignments
   └── s1
📁 data/data/alignments/s1


In [7]:
preprocess_and_save(video_dir='data/data/s1', align_dir='data/data/alignments/s1')


In [8]:
def load_npz(file_path):
    with np.load(file_path) as data:
        return data['frames'], data['alignments']

def tf_data_wrapper(file_path):
    def _load(file):
        f = file.numpy().decode()
        frames, alignments = load_npz(f)
        return frames, alignments

    video, labels = tf.py_function(_load, [file_path], (tf.float32, tf.int64))
    video.set_shape([None, 46, 140, 1])
    labels.set_shape([None])
    return video, labels

def get_datasets(npz_dir='./npz_data', batch_size=8):
    files = tf.data.Dataset.list_files(f"{npz_dir}/*.npz")
    dataset = files.map(tf_data_wrapper, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.shuffle(100)
    dataset = dataset.padded_batch(batch_size, padded_shapes=([None, 46, 140, 1], [None]))
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [9]:
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_len = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_len = tf.cast(tf.shape(y_true)[1], dtype="int64")
    input_len = input_len * tf.ones((batch_len, 1), dtype="int64")
    label_len = label_len * tf.ones((batch_len, 1), dtype="int64")
    return tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_len, label_len)

def build_model():
    inputs = layers.Input(shape=(None, 46, 140, 1))
    x = layers.Conv3D(128, 3, padding='same', activation='relu')(inputs)
    x = layers.MaxPool3D((1, 2, 2))(x)
    x = layers.Conv3D(256, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool3D((1, 2, 2))(x)
    x = layers.Conv3D(75, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool3D((1, 2, 2))(x)
    x = layers.TimeDistributed(layers.Flatten())(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(char_to_num.vocabulary_size() + 1, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss=CTCLoss)
    return model

In [10]:
def get_callbacks():
    checkpoint = ModelCheckpoint('checkpoint.weights.h5', monitor='val_loss', save_weights_only=True)
    def scheduler(epoch, lr): return lr if epoch < 30 else lr * tf.math.exp(-0.1)
    earlystop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    return [checkpoint, LearningRateScheduler(scheduler), earlystop]

In [ ]:
train_data = get_datasets(batch_size=8)
model = build_model()
callbacks = get_callbacks()

model.fit(
    train_data,
    validation_data=train_data,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
 15/125 ━━━━━━━━━━━━━━━━━━━━ 5:27:04 178s/step - loss: 125.6699

In [ ]:
sample_path = './npz_data/bras9a.npz'
frames, labels = load_npz(sample_path)

yhat = model.predict(tf.expand_dims(frames, 0))
decoded = tf.keras.backend.ctc_decode(yhat, input_length=[yhat.shape[1]], greedy=True)[0][0]

predicted = tf.strings.reduce_join(num_to_char(decoded[0])).numpy().decode()
actual = tf.strings.reduce_join(num_to_char(tf.convert_to_tensor(labels))).numpy().decode()

print("✅ Actual:   ", actual)
print("🔮 Predicted:", predicted)